In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from pandas import read_csv
from collections import Counter
from numpy import mean
from numpy import std
from numpy import hstack
from pandas import read_csv
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder as SKlearnOneHot
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.dummy import DummyClassifier
from sklearn.svm import SVC
from sklearn.metrics import auc, classification_report, roc_curve, roc_auc_score
import pickle
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, KFold


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
files=[]
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        files.append(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ml-2024-f/train_final.csv
/kaggle/input/ml-2024-f/test_final.csv


In [2]:
class OneHotEncoder(SKlearnOneHot):
    def __init__(self, **kwargs):
        super(OneHotEncoder, self).__init__(**kwargs)
        self.fit_flag = False

    def fit(self, X, **kwargs):
        out = super().fit(X)
        self.fit_flag = True
        return out

    def transform(self, X, **kwargs):
        sparse_matrix = super(OneHotEncoder, self).transform(X)
        new_columns = self.get_new_columns(X = X)
        d_out = pd.DataFrame(sparse_matrix.toarray(), columns = new_columns, index = X.index)
        return d_out

    def fit_transform(self, X, **kwargs):
        self.fit(X)
        return self.transform(X)

    def get_new_columns(self, X):
        new_columns = []
        for i, column in enumerate(X.columns):
            j = 0
            while j < len(self.categories_[i]):
                new_columns.append(f'{column}_{self.categories_[i][j]}')
                j += 1
        return new_columns

label = 'income>50K'
num_features = ['age', 'capital.gain', 'capital.loss', 'hours.per.week', 'education.num']
cat_features = ['workclass', 'marital.status', 'occupation', 'relationship', 'race', 'sex', 'native.country']

def preprocess_dataframe(df, train=True):
    df = df.drop(['fnlwgt'], axis=1)
    df.drop(['education'], axis = 1, inplace = True)
    df = df.drop(df[df['native.country'] == '?'].index)
    countries = np.array(df['native.country'].unique())
    # print(countries)
    x = np.argwhere(countries=="United-States")
    countries = np.delete(countries, x)
    df['native.country'].replace(countries, 'Other', inplace = True)
    enc = OneHotEncoder()
    encoder_cols = enc.fit_transform(df[cat_features])
    df = pd.concat([pd.concat([df[num_features], encoder_cols], axis=1), df[label]], axis=1)
    return df
    


# print(np.array(train_data['native.country'].unique()))
# print(np.array(test_data['native.country'].unique()))
train_data = pd.read_csv(files[0])
test_data = pd.read_csv(files[1])
df=preprocess_dataframe(train_data)
# df_test = preprocess_dataframe(test_data, False)
# df.isnull().sum()

# print(df['native.country'].value_counts())






features = df.columns.tolist()
features.remove(label)
X = df[features]
y = df[label]


# predictions = .predict(X_test)
# output = pd.DataFrame({'index': X_test.index, 'income >$50K/yr': predictions})
# print("END OF THE PREDICTING")

/tmp/ipykernel_17/4142725209.py:42: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['native.country'].replace(countries, 'Other', inplace = True)


In [3]:

# def trainmodel(nFolds = 2, randomState=2020, features=features, debug= False, model, df = df):
    
#     kf = KFold(n_splits=nFolds, shuffle=True, random_state=randomState)
    
#     # predictions for individual folds
#     predictions = []
    

#     # the index in the original file
#     indices = []
#     true_label=[]

        
#     for train, test in kf.split(df.index.values):
#         if debug:
#             print(model)
#         model.fit(df.iloc[train][features], df.iloc[train][label])

#         preds = model.predict_proba(df.iloc[test][features])[:,1]

#         predictions.append(preds.tolist().copy())

#         indices.append(df.iloc[test].index.tolist().copy())

#         true_label.append(df.iloc[test][label]==1)

#         feature_importance = model.feature_importances_
        
#     return predictions, indices, feature_importance, true_label


# rand_forest = RandomForestClassifier(random_state=2020, n_jobs=-1, n_estimators=100, 
#                                  max_depth=50, max_features=20, min_samples_split=30)

# predictions, indices, feature_importance, true_label = trainmodel(model=rand_forest)
# # for i in range(1,30):
# #     print(predictions[0][i], indices[i], true_label[i])

# # pred_ = np.array(predictions)


# random_forest = {
#     "name":"Random Forest",
#     "predictions": predictions,
#     "indices": indices,
#     "feature importance": feature_importance,
#     "true_label":true_label
# }

# with open("/kaggle/working/random_forest.p", "wb") as fp:
#     pickle.dump(random_forest, fp)

In [4]:
IDs = test_data['ID'].to_numpy()

def preprocess_dataframe_test(df):
    df = df.drop(['fnlwgt'], axis=1)
    df.drop(['education'], axis = 1, inplace = True)
    # df = df.drop(df[df['native.country'] == '?'].index)
    countries = np.array(df['native.country'].unique())
    # print(countries)
    x = np.argwhere(countries=="United-States")
    countries = np.delete(countries, x)
    # print(countries)

    df['native.country'].replace(countries, 'Other', inplace = True)
    enc = OneHotEncoder()
    encoder_cols = enc.fit_transform(df[cat_features])
    df[label]=''
    df = pd.concat([pd.concat([df[num_features], encoder_cols], axis=1), df[label]], axis=1)
    return df

df_test=preprocess_dataframe_test(test_data)

X_test= df_test[features]


/tmp/ipykernel_17/3812567552.py:13: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['native.country'].replace(countries, 'Other', inplace = True)


In [5]:
# with open("/kaggle/working/random_forest.p", "rb") as fp:
#     rand_forest = pickle.load(fp)

model = RandomForestClassifier(random_state=2020, n_jobs=-1, n_estimators=100, 
                                 max_depth=50, max_features=10, min_samples_split=30)
model.fit(X,y)
predictions = model.predict_proba(X_test)[:,1]
output = pd.DataFrame({'ID': IDs, 'Prediction': predictions})
output.to_csv('/kaggle/working/preds10.csv', index=False)

In [6]:

# def predict_test(X_test):
#     model = RandomForestClassifier(random_state=2020, n_jobs=-1, n_estimators=100, 
#                                  max_depth=50, max_features=10, min_samples_split=30)
#     model.fit(X,y)
#     predictions = model.predict_proba(X_test)[:,1]
#     output = pd.DataFrame({'ID': X_test.index, 'Prediction': predictions})
#     output.to_csv('/kaggle/working/income_prediction.csv', index=False)
#     return output